# `__prepare__` Method

we know that when we are creating the class , the metaclass `__new__` method invoked with an argument (cls_dict) for thr class dictionary.

In [1]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('MyMeta.__new__ called...')
        print('\tcls: ', mcls, type(mcls))
        print('\tname:', name, type(name))
        print('\tbases: ', bases, type(bases))
        print('\tcls_dict:', cls_dict, type(cls_dict))
        print('\tkwargs:', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [2]:
class MyClass(metaclass=MyMeta):
    pass

MyMeta.__new__ called...
	cls:  <class '__main__.MyMeta'> <class 'type'>
	name: MyClass <class 'str'>
	bases:  () <class 'tuple'>
	cls_dict: {'__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
	kwargs: {}


So, as we see, cls_dict is a dictionary and it also contains some information already. It is obviously being created somewhere before being passed to the __new__ method.

The class dictionary is actually created by calling the __prepare__ method, which the type class implements.

When the class is created, Python calls __prepare__ and uses the return value of that method as the initialized class dictionary. Then right before calling __new__ it adds a few items into that dictionary, and then calls the __new__ method using that pre-created and initialized dictionary.

Since __prepare__ is just a method in type, we can override it.

In [4]:
class MyMeta(type):
    @staticmethod
    def __prepare__(name, bases, **kwargs):
        print('MyMeta.__prepare__ called...')
        print('\tname:', name)
        print('\tkwargs:', kwargs)
        return {'a': 100, 'b': 200}

    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('MyMeta.__new__ called...')
        print('\tcls: ', mcls, type(mcls))
        print('\tname:', name, type(name))
        print('\tbases: ', bases, type(bases))
        print('\tcls_dict:', cls_dict, type(cls_dict))
        print('\tkwargs:', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)


In [5]:
class MyClass(metaclass=MyMeta, kw1=10, kw2=20):
    pass

MyMeta.__prepare__ called...
	name: MyClass
	kwargs: {'kw1': 10, 'kw2': 20}
MyMeta.__new__ called...
	cls:  <class '__main__.MyMeta'> <class 'type'>
	name: MyClass <class 'str'>
	bases:  () <class 'tuple'>
	cls_dict: {'a': 100, 'b': 200, '__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
	kwargs: {'kw1': 10, 'kw2': 20}


Notice how the __prepare__ method was called before the __new__ method was called.

Also notice how it contains the items 'a': 100 and 'b': 200 which we injected in the __prepare__ method.

The cls_dict argument in __new__ has a couple of extra items that it injects for us prior to calling the __new__ method.

Of course, if we do not specify a __prepare__ method in our metaclass, we inherit the one that is already defined in type - which returns an empty dictionary.

In [6]:
type.__prepare__()

{}

In [8]:
class CustomDict(dict):
    def __setitem__(self, key, value):
        print(f'Setting {key} = {value} in custom dictionary')
        super().__setitem__(key, value)

    def __getitem__(self, key):
        print(f'Getting {key} from custom dictionary')
        return int(super().__getitem__(key))

In [9]:
class MyMeta(type):
    def __prepare__(name, bases):
        return CustomDict()

    def __new__(mcls, name, bases, cls_dict):
        print('metaclass __new__ called...')
        print(f'\ttype(cls_dict) = {type(cls_dict)}')
        print(f'\tcls_dict={cls_dict}')

In [10]:
class MyClass(metaclass=MyMeta):
    pass

Getting __name__ from custom dictionary
Setting __module__ = __main__ in custom dictionary
Setting __qualname__ = MyClass in custom dictionary
metaclass __new__ called...
	type(cls_dict) = <class '__main__.CustomDict'>
	cls_dict={'__module__': '__main__', '__qualname__': 'MyClass'}


As you can see, the dictionary we returned from __prepare__ was a CustomDict instance that is eventually passed to __new__ when it is called.

And between __prepare__ and __new__, Python accessed our dictionary to read/write a few items.